In [14]:

import pandas as pd 
import numpy as np
import seaborn as sns
from sklearn.ensemble import StackingClassifier
# from sklearn.datasets import make_classification
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold,RepeatedStratifiedKFold
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import StackingRegressor
import warnings
warnings.filterwarnings("ignore")
from lightgbm import LGBMClassifier
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LogisticRegression
import scipy.stats as st
import sklearn.metrics as metrics
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split,KFold,StratifiedKFold,GridSearchCV,RandomizedSearchCV,cross_val_score
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import r2_score,roc_auc_score,recall_score,classification_report,mean_squared_error,accuracy_score,f1_score,confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/petadoption-mod/test_mod.csv
/kaggle/input/petadoption-mod/train_mod.csv


In [15]:
train=pd.read_csv("../input/petadoption-mod/train_mod.csv")
test=pd.read_csv("../input/petadoption-mod/test_mod.csv")



#label encoding the features for the modelling process
le=LabelEncoder()
le.fit(train["color_type"])
# train["color_type"]=le.fit_transform(train["color_type"])
train["color_type"]=le.transform(train["color_type"])
test["color_type"]=le.transform(test["color_type"])

In [16]:
train_set,val_set=train_test_split(train,test_size=.25,random_state=42)
train_set.shape,val_set.shape

#-------------------------------------------------------------------------------
#train_set
train_data=train_set.drop(['pet_id', 'issue_date', 'listing_date','breed_category', 'pet_category','days_stayed']
                          ,axis=1)
train_res_breed=train_set.breed_category
train_res_pet=train_set.pet_category

#-------------------------------------------------------------------------------
#val_set
val_data=val_set.drop(['pet_id', 'issue_date', 'listing_date','breed_category', 'pet_category','days_stayed'],
                     axis=1)
val_res_breed=val_set.breed_category
val_res_pet=val_set.pet_category
#-------------------------------------------------------------------------------
train_full=train.drop(['pet_id', 'issue_date', 'listing_date','breed_category', 'pet_category','days_stayed']
                          ,axis=1)
train_full_breed=train.breed_category
train_full_pet=train.pet_category
test_full=test.drop(['pet_id', 'issue_date', 'listing_date','days_stayed']
                          ,axis=1)

#-------------------------------------------------------------------------------

train_data.shape,train_res_breed.shape,val_data.shape,val_res_breed.shape,train_full.shape,test_full.shape

((14125, 9), (14125,), (4709, 9), (4709,), (18834, 9), (8072, 9))

In [17]:
lgbc=LGBMClassifier()

In [18]:
models=[lgbc]
sctr,scte,auc,ps,rs=[],[],[],[],[]
def ensemble_for_train_val(X_train,y_train,X_test, y_test):
    for model in models:
        print(model.__class__.__name__)
        model.fit(X_train, y_train)
        y_test_pred = model.predict(X_test)
        y_test_pred_new=model.predict_proba(X_test)
        y_test_pred_new=y_test_pred_new[:,1]
        train_score=model.score(X_train,y_train)
        test_score=model.score(X_test,y_test)
        p_score=metrics.precision_score(y_test,y_test_pred,average='micro')
        r_score=metrics.recall_score(y_test,y_test_pred,average='micro')
        
        sctr.append(train_score)
        scte.append(test_score)
        ps.append(p_score)
        rs.append(r_score)
    return sctr,scte,ps,rs




In [19]:

ensemble_for_train_val(train_data,train_res_breed, val_data, val_res_breed)
# 'ab_rf','ab_dt','ab_nb','ab_lr','bgcl_lr'

ensemble=pd.DataFrame({'names':["lgbc"],
                       'training':sctr,'testing':scte,'precision':ps,'recall':rs,})

ensemble=ensemble.sort_values(by='precision',ascending=False).reset_index(drop=True)
ensemble

LGBMClassifier


,names,training,testing,precision,recall
0,lgbc,0.976212,0.901253,0.901253,0.901253


In [20]:
help(GridSearchCV)

Help on class GridSearchCV in module sklearn.model_selection._search:

class GridSearchCV(BaseSearchCV)
 |  GridSearchCV(estimator, param_grid, *, scoring=None, n_jobs=None, iid='deprecated', refit=True, cv=None, verbose=0, pre_dispatch='2*n_jobs', error_score=nan, return_train_score=False)
 |  
 |  Exhaustive search over specified parameter values for an estimator.
 |  
 |  Important members are fit, predict.
 |  
 |  GridSearchCV implements a "fit" and a "score" method.
 |  It also implements "predict", "predict_proba", "decision_function",
 |  "transform" and "inverse_transform" if they are implemented in the
 |  estimator used.
 |  
 |  The parameters of the estimator used to apply these methods are optimized
 |  by cross-validated grid-search over a parameter grid.
 |  
 |  Read more in the :ref:`User Guide <grid_search>`.
 |  
 |  Parameters
 |  ----------
 |  estimator : estimator object.
 |      This is assumed to implement the scikit-learn estimator interface.
 |      Either e

In [21]:
help(lgbc)


Help on LGBMClassifier in module lightgbm.sklearn object:

class LGBMClassifier(LGBMModel, sklearn.base.ClassifierMixin)
 |  LGBMClassifier(boosting_type='gbdt', num_leaves=31, max_depth=-1, learning_rate=0.1, n_estimators=100, subsample_for_bin=200000, objective=None, class_weight=None, min_split_gain=0.0, min_child_weight=0.001, min_child_samples=20, subsample=1.0, subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0, reg_lambda=0.0, random_state=None, n_jobs=-1, silent=True, importance_type='split', **kwargs)
 |  
 |  LightGBM classifier.
 |  
 |  Method resolution order:
 |      LGBMClassifier
 |      LGBMModel
 |      sklearn.base.BaseEstimator
 |      sklearn.base.ClassifierMixin
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  fit(self, X, y, sample_weight=None, init_score=None, eval_set=None, eval_names=None, eval_sample_weight=None, eval_class_weight=None, eval_init_score=None, eval_metric=None, early_stopping_rounds=None, verbose=True, feature_name='auto', cat

In [22]:
def prec(x,y):
    prec=metrics.precision_score(x,y,average='micro')
    return prec

In [23]:

# ensemble_for_train_val(train_data,train_res_pet, val_data, val_res_pet)
# # 'ab_rf','ab_dt','ab_nb','ab_lr','bgcl_lr'

# ensemble=pd.DataFrame({'names':["lgbc"],
#                        'training':sctr,'testing':scte,'precision':ps,'recall':rs,})

# ensemble=ensemble.sort_values(by='precision',ascending=False).reset_index(drop=True)
# ensemble

param_dict={
    'boosting_type' :["dart"],
    'is_unbalance':[True],
    'n_estimators':[20,30,50],
    'num_leaves':[8,10,12], 
    'max_depth':[None,8,10,12],
#     'feature_fraction':[0.5,.75],  
#     'bagging_fraction':[0.8,.5], 
#     'bagging_freq':[15,20], 
    'learning_rate':[.1]
}

gridSearchCV = GridSearchCV(estimator = lgbc, 
    param_grid = param_dict, 
    scoring='accuracy',
    n_jobs=-1,
    iid=False, 
    verbose=1,
    cv=3)
gridSearchCV.fit(train_full,train_full_breed)


Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    6.5s
[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed:   14.0s finished


GridSearchCV(cv=3, estimator=LGBMClassifier(), iid=False, n_jobs=-1,
             param_grid={'boosting_type': ['dart'], 'is_unbalance': [True],
                         'learning_rate': [0.1], 'max_depth': [None, 8, 10, 12],
                         'n_estimators': [20, 30, 50],
                         'num_leaves': [8, 10, 12]},
             scoring='accuracy', verbose=1)

In [24]:
 gridSearchCV.best_params_, gridSearchCV.best_score_

({'boosting_type': 'dart',
  'is_unbalance': True,
  'learning_rate': 0.1,
  'max_depth': None,
  'n_estimators': 30,
  'num_leaves': 8},
 0.9052245938196878)

In [26]:

# ensemble_for_train_val(train_data,train_res_pet, val_data, val_res_pet)
# # 'ab_rf','ab_dt','ab_nb','ab_lr','bgcl_lr'

# ensemble=pd.DataFrame({'names':["lgbc"],
#                        'training':sctr,'testing':scte,'precision':ps,'recall':rs,})

# ensemble=ensemble.sort_values(by='precision',ascending=False).reset_index(drop=True)
# ensemble

param_dict={
    'boosting_type' :["dart"],
    'is_unbalance':[True],
    'n_estimators':[400,500,600],
    'num_leaves':[10,20], 
    'max_depth':[8,10,12],
#     'feature_fraction':[0.5,.75],  
#     'bagging_fraction':[0.8,.5], 
#     'bagging_freq':[15,20], 
    'learning_rate':[0.01,.1]
}

gridSearchCV = GridSearchCV(estimator = lgbc, 
    param_grid = param_dict, 
    scoring='accuracy',
    n_jobs=-1,
    iid=False, 
    verbose=1,
    cv=3)
gridSearchCV.fit(train_full,train_full_pet)
gridSearchCV.best_params_, gridSearchCV.best_score_

Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  8.1min
[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed: 22.9min finished


({'boosting_type': 'dart',
  'is_unbalance': True,
  'learning_rate': 0.1,
  'max_depth': 12,
  'n_estimators': 600,
  'num_leaves': 20},
 0.9047998300945098)

In [ ]:
lgbc=LGBMClassifier(boosting_type= 'dart',is_unbalance= True,learning_rate= 0.1,max_depth= None,n_estimators= 30,num_leaves= 8)
lgbc.fit(train_full,train_full_breed)
y_test_pred_breed = lgbc.predict(test_full)

lgbc=LGBMClassifier(boosting_type= 'dart',is_unbalance=True,learning_rate= 0.1,max_depth= 12,n_estimators= 600,num_leaves= 20)
lgbc.fit(train_full,train_full_breed)
y_test_pred2_pet=lgbc.predict(test_full)
id=pd.Series(test["pet_id"])
submission=pd.DataFrame({'pet_id':id,'breed_category':y_test_pred_breed,'pet_category':y_test_pred2_pet})
submission.to_csv("pet_doption_hyperparametertuned.csv",index=False)